<a href="https://colab.research.google.com/github/likhith-t-25/human-activity-recognition/blob/main/har%20with%20gui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install -q gradio seaborn

import gradio as gr
import pandas as pd
import numpy as np
import io
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

def classify_sensor_data(file):
    try:
        df = pd.read_csv(file.name)
    except Exception as e:
        return f"❌ Error reading file: {e}", None, None

    # Detect activity column
    activity_column = None
    for col in df.columns:
        if 'activity' in col.lower() or 'label' in col.lower():
            activity_column = col
            break
    if activity_column is None:
        return "❌ No 'activity' or 'label' column found in CSV.", None, None

    # Fill missing values
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            df[col].fillna(df[col].median(), inplace=True)
        else:
            df[col].fillna(df[col].mode()[0], inplace=True)

    X = df.drop(activity_column, axis=1)
    y = df[activity_column]

    # Encode and scale
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Dynamic test size handling
    num_classes = len(np.unique(y_encoded))
    min_test_size = num_classes / len(df)
    test_size = max(0.2, min_test_size)

    try:
        X_train, X_test, y_train, y_test = train_test_split(
            X_scaled, y_encoded, test_size=test_size, random_state=42, stratify=y_encoded
        )
    except ValueError as e:
        return f"❌ Error splitting data: {e}", None, None

    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, target_names=label_encoder.classes_)
    summary = f"✅ Accuracy: {acc * 100:.2f}%\n\n{report}"

    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    cm_buf = io.BytesIO()
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.tight_layout()
    plt.savefig(cm_buf, format='png')
    plt.close()
    cm_buf.seek(0)
    cm_img = Image.open(cm_buf)

    # Feature Importance
    feat_buf = io.BytesIO()
    feat_df = pd.DataFrame({'Feature': X.columns, 'Importance': model.feature_importances_})
    feat_df = feat_df.sort_values(by='Importance', ascending=False)
    plt.figure(figsize=(8, 6))
    sns.barplot(x='Importance', y='Feature', data=feat_df)
    plt.title("Feature Importance")
    plt.tight_layout()
    plt.savefig(feat_buf, format='png')
    plt.close()
    feat_buf.seek(0)
    feat_img = Image.open(feat_buf)

    return summary, cm_img, feat_img

# Launch Gradio GUI
gr.Interface(
    fn=classify_sensor_data,
    inputs=gr.File(label="📁 Upload Sensor CSV"),
    outputs=[
        gr.Textbox(label="📋 Classification Report"),
        gr.Image(label="📊 Confusion Matrix", type="pil"),
        gr.Image(label="🔍 Feature Importance", type="pil")
    ],
    title="Sensor-Based Human Activity Recognition (HAR)",
    description="Upload a CSV file containing gyroscope + accelerometer data to classify human activities using Random Forest."
).launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://32413bdb5a7e11cecb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
